Jupyter Notebook to run tl-rnn stepwise

In [1]:
    from util import *
    import pickle
    import pandas as pd
    import random
    from model import tlrnn
    from dataPrep import dataprep
    import pickle

Using TensorFlow backend.


finished


In [2]:
"""
define directory where you stored the data
"""
directory = "~/tl_rnn/data/"

In [3]:
"""
read training dataset and get rid of NA's
"""
df = pd.read_csv(directory + "sequential_data_train.csv", verbose=True)
df = df.fillna(0)

Tokenization took: 21.90 ms
Type conversion took: 13.64 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.41 ms
Type conversion took: 12.72 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.84 ms
Type conversion took: 13.61 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.33 ms
Type conversion took: 14.40 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.32 ms
Type conversion took: 14.34 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.32 ms
Type conversion took: 14.31 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.05 ms
Type conversion took: 15.03 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.56 ms
Type conversion took: 13.78 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.26 ms
Type conversion took: 16.10 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 11.26 ms
Type conversion took: 15.39 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.27 ms
Type conversion took: 15

In [4]:
"""
create dataprep object. in this object all hyperparameters for dataprocessing and sampling are stored
please double check all parameters in config.py before running it!!!
"""

dataobj = dataprep(dataframe=df, id_name="user_ID", sequence_id = "sequence_ID")

DATAFRAME COLUMNS  ['user_ID', 'sequence_ID', 'eventData_1', 'eventData_2', 'eventData_3']
SEQ_LEN_MAX 100
TOP 20 Categories of all Covariates ['cat_45', 'cat_73', 'cat_44', 'cat_26', 'cat_90', 'cat_213', 'cat_160', 'cat_69', 'cat_244', 'cat_232', 'cat_752', 'cat_133', 'cat_506', 'cat_672', 'cat_425', 'cat_403', 'cat_235', 'cat_390', 'cat_83', 'cat_547']
COVARIATES_ALL ['eventData_1', 'eventData_2', 'eventData_3']
SAMPLES PER USER 128
SEQUENCE LENGTH MAX 100


In [5]:
"""
        move data from original dataframe into lists of arrays,
        encode them properly: categories with a ranked based encoding
        one list entry per user array, with [timeframe index (e.g. nth week), seq_len(e.g. 1000), feature]
        time order conserved (t-1,t0,t+1)

        params: only members coming from dataprep init

        input:  columnwise dataframe read by dataframe() init

        return: list with each element being a user nested with another list of single array sequences:
                [user][array[timestep, features]]
"""

dataobj.preprocessor()

dataframe -> tensor ...
--------------------------------------------------
16/02/2022 13:55:41
0 of 9000 users 
--------------------------------------------------
16/02/2022 13:56:58
1000 of 9000 users 
--------------------------------------------------
16/02/2022 13:58:09
2000 of 9000 users 
--------------------------------------------------
16/02/2022 13:59:08
3000 of 9000 users 
--------------------------------------------------
16/02/2022 13:59:57
4000 of 9000 users 
--------------------------------------------------
16/02/2022 14:00:37
5000 of 9000 users 
--------------------------------------------------
16/02/2022 14:01:09
6000 of 9000 users 
--------------------------------------------------
16/02/2022 14:01:35
7000 of 9000 users 
--------------------------------------------------
16/02/2022 14:01:56
8000 of 9000 users 
tensor builder finished


In [6]:
"""
evaluate cardinalities of input values
this numbers will be taken into account to dimension the neural network model
"""

dataobj.build_cardinality()

{'eventData_1': 34808, 'eventData_2': 26, 'eventData_3': 33}


Before training with the training data, let's prepare the holdout (test) data-set to evaluate the resulting model

In [7]:
"""
read column-like TEST dataframe and get rid of NA's
"""

df_test = pd.read_csv(directory + "sequential_data_test.csv")
df_test = df_test.fillna(0)

In [8]:
"""
create dataprep object for test data
"""

dataobj_test = dataprep(dataframe=df_test, id_name="user_ID", sequence_id = "sequence_ID")

DATAFRAME COLUMNS  ['user_ID', 'sequence_ID', 'eventData_1', 'eventData_2', 'eventData_3']
SEQ_LEN_MAX 100
TOP 20 Categories of all Covariates ['cat_45', 'cat_73', 'cat_44', 'cat_90', 'cat_26', 'cat_69', 'cat_213', 'cat_232', 'cat_752', 'cat_160', 'cat_244', 'cat_251', 'cat_390', 'cat_672', 'cat_547', 'cat_324', 'cat_425', 'cat_506', 'cat_133', 'cat_402']
COVARIATES_ALL ['eventData_1', 'eventData_2', 'eventData_3']
SAMPLES PER USER 128
SEQUENCE LENGTH MAX 100


In [9]:
"""
VERY IMPORTANT!!

Overwrite the feature encodings (rank-based) with the ones from the training set
Otherwise training and test are encoded differently and the predicitve power gets bad
"""

dataobj_test.ranks = dataobj.ranks

In [10]:
"""
run preprocessor and bring datframe into propper list-like data structure for sampler
"""
dataobj_test.preprocessor()

dataframe -> tensor ...
--------------------------------------------------
16/02/2022 14:02:13
0 of 1000 users 
tensor builder finished


In [11]:
"""
Sample triplets from holdout (test) set to test re-identification score
Notice: In training mode, the triplets are sampled online during training,
for each epoch a new set of triplets
"""

dataobj_test.sampler(triplets_per_user=8, users_per_file="all")

triplets per user:  8
0 of 1000 users 


Let's train the network:

In [12]:
"""
define cells and build tl-rnn keras model
put in dataobj, where also the pre-processed data-set
is included for "online" generator sampling
"""

my_tlrnn = tlrnn(cells=128, dataobj=dataobj)
my_tlrnn.build()

COVARIATES:  {'eventData_1': 34808, 'eventData_2': 26, 'eventData_3': 33}
SEQUENCE LEN MAX 100
CELLS 128
PATIENCE 4
BATCH SIZE 64
BATCHES PER USER 2
OPTIMIZER <keras.optimizers.Adam object at 0x7f2e3dcca2b0>





In [13]:
"""
alpha: seperation strength (regularizer) for triplet seperation
too much: model doesn't generalize well
too weak: model doesn't discriminate well
start with alpha=1.0 and work upwards
"""

alpha=1.0
run_name = "my_run_01"

In [14]:
"""
generator model: draws samples from dataobj automatically
define test_set manually from test set above for re-identification evaluation on hold-out
"""

my_tlrnn.train_generator(run_name=run_name, alpha=alpha, test_set=dataobj_test.triplet_tensor)

split training and validation set
length training set:  8550
length validation set:  450
compile model...

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 100, 1)       0                                            
__










Epoch 1/12
17100/17100 [==============================] - 361s 21ms/step - loss: 0.4211 - val_loss: 0.3201

score L1:  0.8635
Epoch 2/12
17100/17100 [==============================] - 357s 21ms/step - loss: 0.2465 - val_loss: 0.2901
score L1:  0.880875
Epoch 3/12
17100/17100 [==============================] - 367s 21ms/step - loss: 0.1809 - val_loss: 0.2697
score L1:  0.8855
Epoch 4/12
17100/17100 [==============================] - 342s 20ms/step - loss: 0.1415 - val_loss: 0.2596
score L1:  0.894125
Epoch 5/12
17100/17100 [==============================] - 374s 22ms/step - loss: 0.1131 - val_loss: 0.2417
score L1:  0.902125
Epoch 6/12
17100/17100 [==============================] - 397s 23ms/step - loss: 0.0936 - val_loss: 0.2368
score L1:  0.90425
Epoch 7/12
17100/17100 [==============================] - 398s 23ms/step - loss: 0.0804 - val_loss: 0.2354
score L1:  0.901375
Epoch 8/12
17100/17100 [==============================] - 398s 23ms/step - loss: 0.0709 - val_loss: 0.2375

In [65]:
## Here is how the model can be used to embedd a single sequence:

# take single sequence out of test data set for example:
single_seq = dataobj_test.data_list[0][0]

if len(single_seq[:,0]) > my_tlrnn.sequence_len_max:
    raise Exception("Input sequence is longer than seq-len-max. Shorten the sequence and repeat.")

# zero-padding for the sequence to bring it to the nominal sequence length
# first dim is important, but for the single sequence case just 1
seq = np.zeros((1, dataobj.sequence_len_max, my_tlrnn.cov_num), dtype='int32')
seq[0, :len(single_seq[:,0]), :] = single_seq[:, :]

# translate sequence into embedding vector with the trained tl_rnn model
# model requires list of features
single_embedd = my_tlrnn.model_pred.predict([seq[:, :, (i):(i+1)] for i in list(range(my_tlrnn.cov_num))])

In [66]:
print(single_embedd)

[[-0.12670484  0.01557162  0.07797565 -0.05971112 -0.4180501   0.07241513
   0.01756571 -0.15987296 -0.03673608  0.06769748 -0.00820197 -0.199908
   0.06610264  0.03982965  0.11915255  0.03439994 -0.04824077 -0.08312859
   0.02021081 -0.05244791 -0.04547481 -0.00708578 -0.03062822  0.04037116
   0.05492684 -0.01577551 -0.11857747  0.12923999  0.00644235  0.15515679
   0.08415571  0.06526165  0.04574451  0.06990935  0.06214139  0.04033377
  -0.09651881 -0.07461414  0.14476398  0.11885166  0.04625008  0.09086294
  -0.21538562  0.04252753 -0.01896777  0.06049617 -0.00833463 -0.0458732
   0.02249925 -0.01404572  0.02047411  0.1769096  -0.04371607 -0.00906524
   0.09585367 -0.03242134  0.02680962  0.02872449 -0.01748445 -0.22998078
  -0.00192439 -0.12011491  0.03149238 -0.06925411 -0.0511024  -0.05010792
   0.03355582 -0.03059018 -0.21594599 -0.09640229  0.00950254 -0.03980622
  -0.09664448 -0.14582826  0.10067647  0.13773139 -0.07426036  0.07493069
  -0.04835485 -0.08211111 -0.07620607  0.

If you are not happy with the accuracy, tune hyperparameters for better results. Increasing alpha can help.

END OF SCRIPT